> In this notebook the models are tested on the MPIL dataset. Required libraries and files from the dataset are imported. A separate DataFrame has been compiled for each of the two testing conditions in the MPIL study &ndash; `dfeo` for eyes open, and `dfec` for eyes closed.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import joblib
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.constraints import max_norm
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
dfeo = pd.read_csv('EEG_MPIL_dataset_powers_EO.csv')
dfec = pd.read_csv('EEG_MPIL_dataset_powers_EC.csv')
dfsd = pd.read_csv('MPIL_subject_data.csv')

In [3]:
dfeo

,Unnamed: 0,Fp1a delta,Fp1a theta,Fp1a alpha,Fp1a beta,Fp1a gamma,Fp2a delta,Fp2a theta,Fp2a alpha,Fp2a beta,...,P3/P4 delta,P3/P4 theta,P3/P4 alpha,P3/P4 beta,P3/P4 gamma,O1/O2 delta,O1/O2 theta,O1/O2 alpha,O1/O2 beta,O1/O2 gamma
0,10002,2.963294,1.758861,2.741883,14.882855,4.597735,3.045786,1.772866,2.374005,7.508171,...,0.473699,0.487901,0.505614,0.567835,0.557093,0.465475,0.468386,0.537042,0.543826,0.463206
1,10003,6.513888,5.829570,14.301176,4.517712,0.849736,6.655006,5.808280,13.932360,3.753285,...,0.485737,0.458422,0.443813,0.466147,0.500808,0.509675,0.498215,0.478986,0.532515,0.634469
2,10004,4.533997,4.103808,8.202011,10.934104,3.468158,2.508404,2.685708,4.977986,7.419088,...,0.536361,0.531348,0.536021,0.507245,0.510665,0.506654,0.492829,0.484946,0.462731,0.474542
3,10005,13.001012,6.376373,8.479070,10.935127,1.120146,10.528730,5.092280,7.257407,9.555641,...,0.458471,0.484618,0.518575,0.480005,0.473664,0.502523,0.514538,0.537265,0.500549,0.485857
4,10006,2.751090,2.157247,4.778439,7.531138,2.043466,2.998565,2.410071,5.312022,11.679908,...,0.528464,0.519394,0.478897,0.516000,0.495676,0.523968,0.515604,0.505154,0.516178,0.496556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,10315,3.508636,2.245006,2.866946,3.807370,0.418680,4.266233,2.469076,3.211018,4.369106,...,0.469807,0.493699,0.523106,0.478229,0.527235,0.465137,0.481502,0.487168,0.478506,0.496092
166,10316,5.668383,4.374410,2.166326,1.630367,0.352707,6.157412,4.673679,2.312181,1.822349,...,0.496886,0.504561,0.494965,0.491063,0.470268,0.501390,0.504902,0.496936,0.495394,0.482235
167,10318,4.370899,2.288354,1.699240,5.647237,0.892734,6.248704,2.952553,2.143792,7.067242,...,0.461284,0.459103,0.472405,0.458490,0.470130,0.497453,0.485832,0.499386,0.501177,0.508355
168,10319,12.263410,4.861399,13.168760,14.862353,1.869617,11.409593,4.538210,13.734878,10.770214,...,0.454806,0.483272,0.509207,0.443675,0.472377,0.494690,0.491955,0.558415,0.510408,0.507811


In [4]:
dfec

,Unnamed: 0,Fp1a delta,Fp1a theta,Fp1a alpha,Fp1a beta,Fp1a gamma,Fp2a delta,Fp2a theta,Fp2a alpha,Fp2a beta,...,P3/P4 delta,P3/P4 theta,P3/P4 alpha,P3/P4 beta,P3/P4 gamma,O1/O2 delta,O1/O2 theta,O1/O2 alpha,O1/O2 beta,O1/O2 gamma
0,10002,2.475371,1.638637,3.015637,11.598780,3.483368,2.282586,1.552220,2.664108,7.493583,...,0.478741,0.500657,0.550677,0.566536,0.580211,0.495397,0.495227,0.520955,0.557818,0.472947
1,10003,19.922156,21.127609,39.019259,6.427448,0.734595,21.073421,22.050736,37.863523,6.997013,...,0.467111,0.462132,0.490572,0.481299,0.490050,0.480509,0.494900,0.492222,0.510392,0.615860
2,10004,7.441585,7.143337,11.554174,6.022341,0.700233,7.537999,6.685701,9.016543,6.249960,...,0.523174,0.535934,0.554881,0.513354,0.509120,0.502332,0.495862,0.464250,0.479605,0.473999
3,10005,9.808763,6.968096,11.118838,10.138833,0.358580,9.058049,6.610697,10.778979,9.745145,...,0.465592,0.500293,0.470638,0.482173,0.483798,0.516039,0.523242,0.544121,0.498107,0.504212
4,10006,3.142933,4.254568,8.852245,7.044991,0.569409,3.266391,4.244158,8.766980,7.114891,...,0.515464,0.507248,0.411856,0.471365,0.498517,0.514724,0.508148,0.471631,0.503539,0.503567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,10316,4.775194,3.587250,3.626048,1.975405,0.432544,5.345416,3.560776,4.097011,4.051232,...,0.470399,0.479859,0.370586,0.459771,0.481945,0.513476,0.508598,0.381715,0.447990,0.487156
172,10317,1.748817,1.188473,2.273615,1.128040,0.296815,2.042888,1.330412,2.427322,1.010907,...,0.448729,0.444600,0.402036,0.434526,0.475279,0.512463,0.512346,0.489442,0.532473,0.602015
173,10318,7.095514,3.906354,4.303735,6.297427,0.758554,8.317032,4.403639,4.701117,7.391188,...,0.453108,0.409012,0.343040,0.410707,0.478306,0.511121,0.488979,0.555404,0.515656,0.510222
174,10319,10.918286,5.050628,23.635058,10.075020,0.655482,11.342959,5.360507,27.146225,11.444452,...,0.480541,0.488522,0.421483,0.440575,0.439214,0.517001,0.515320,0.514562,0.506926,0.504901


In [5]:
dfsd

,ID,Gender_ 1=female_2=male,Age,Handedness,Education,DRUG,DRUG_0=negative_1=Positive,Unnamed: 7,Smoking,"Smoking_num_(Non-smoker=1, Occasional Smoker=2, Smoker=3)",...,SKID_Diagnoses 1,SKID_Diagnoses 2,Comments_SKID_assessment,Hamilton_Scale,BSL23_sumscore,BSL23_behavior,AUDIT,Standard_Alcoholunits_Last_28days,Alcohol_Dependence_In_1st-3rd_Degree_relative,Relationship_Status
0,sub-010017,2,20-25,right,Gymnasium,negative,0.0,NaN,non-smoker,1.0,...,#CODE:V71.09 #DESC:No Diagnosis or Condition ...,NaN,NaN,0.0,NaN,NaN,12.0,47,No,No
1,sub-010015,2,20-25,right,Gymnasium,negative,0.0,NaN,occasional smoker,2.0,...,#CODE:V71.09 #DESC:No Diagnosis or Condition ...,NaN,NaN,0.0,NaN,NaN,12.0,45,Yes,Yes
2,sub-010100,2,25-30,right,Gymnasium,negative,0.0,NaN,occasional smoker,2.0,...,NaN,NaN,NaN,2.0,NaN,NaN,0.0,12,unknown,Yes
3,sub-010003,1,20-25,right,Gymnasium,negative,0.0,NaN,non-smoker,1.0,...,#CODE:V71.09 #DESC:No Diagnosis or Condition ...,NaN,NaN,3.0,NaN,NaN,1.0,0,No,No
4,sub-010022,2,20-25,right,Gymnasium,negative,0.0,NaN,non-smoker,1.0,...,#CODE:V71.09 #DESC:No Diagnosis or Condition ...,NaN,NaN,2.0,NaN,NaN,7.0,22,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,sub-010005,2,25-30,right,Gymnasium,negative,0.0,NaN,non-smoker,1.0,...,#CODE:V71.09 #DESC:No Diagnosis or Condition ...,NaN,NaN,0.0,4.0,0.0,1.0,1,No,No
223,sub-010231,2,20-25,left,Gymnasium,negative,0.0,NaN,non-smoker,1.0,...,#CODE:V71.09 #DESC:No Diagnosis or Condition ...,NaN,"suspected diagnosis: panic disorder, alcohol a...",1.0,4.0,1.0,5.0,18,no,no
224,sub-010232,2,25-30,left,Gymnasium,negative,0.0,NaN,non-smoker,1.0,...,#CODE:V71.09 #DESC:No Diagnosis or Condition ...,NaN,NaN,5.0,2.0,1.0,3.0,29,no,yes
225,sub-010233,2,25-30,right,Gymnasium,negative,0.0,NaN,non-smoker,1.0,...,#CODE:300.29 #DESC: Mild Specific phobia #SPEC...,NaN,subclinical specific phobia,3.0,3.0,1.0,3.0,"16,5",no,yes


> The AUDIT screen is a questionnaire on alcohol use that generates a score ranging from 0 to 40. A score of 8 or above indicates hazardous or risky drinking, so this has been used as a cut-off for dividing the participant group into hazardous drinkers and others. In the cell below a label column is added to the `dfeo` and `dfec` DataFrames, showing "1" for hazardous drinkers and "0" for others. Any subjects not appearing in all 3 DataFrames are dropped from the analysis.

In [6]:
dfeo['label'] = [0] * len(dfeo.index)
dfec['label'] = [0] * len(dfec.index)
for sub_id in dfeo['Unnamed: 0']:
    if not sub_id in dfec['Unnamed: 0'].values:
        dfeo = dfeo.drop(dfeo.index[dfeo['Unnamed: 0'] == sub_id].item())
    else:
        if not "sub-0" + str(int(sub_id)) in dfsd['ID'].values:
            dfeo = dfeo.drop(dfeo.index[dfeo['Unnamed: 0'] == sub_id].item())
            dfec = dfec.drop(dfeo.index[dfec['Unnamed: 0'] == sub_id].item())
        else:
            idxsd = dfsd.index[dfsd['ID'] == "sub-0" + str(int(sub_id))].item()
            idxeo = dfeo.index[dfeo['Unnamed: 0'] == sub_id].item()
            idxec = dfec.index[dfec['Unnamed: 0'] == sub_id].item()
            label = int(dfsd.loc[idxsd]['AUDIT'] >= 8.0)
            dfeo.loc[idxeo, 'label'] = label
            dfec.loc[idxec, 'label'] = label
for sub_id in dfec['Unnamed: 0']:
    if not sub_id in dfeo['Unnamed: 0'].values:
        dfec = dfec.drop(dfec.index[dfec['Unnamed: 0'] == sub_id].item())

In [7]:
dfeo

,Unnamed: 0,Fp1a delta,Fp1a theta,Fp1a alpha,Fp1a beta,Fp1a gamma,Fp2a delta,Fp2a theta,Fp2a alpha,Fp2a beta,...,P3/P4 theta,P3/P4 alpha,P3/P4 beta,P3/P4 gamma,O1/O2 delta,O1/O2 theta,O1/O2 alpha,O1/O2 beta,O1/O2 gamma,label
0,10002,2.963294,1.758861,2.741883,14.882855,4.597735,3.045786,1.772866,2.374005,7.508171,...,0.487901,0.505614,0.567835,0.557093,0.465475,0.468386,0.537042,0.543826,0.463206,0
1,10003,6.513888,5.829570,14.301176,4.517712,0.849736,6.655006,5.808280,13.932360,3.753285,...,0.458422,0.443813,0.466147,0.500808,0.509675,0.498215,0.478986,0.532515,0.634469,0
2,10004,4.533997,4.103808,8.202011,10.934104,3.468158,2.508404,2.685708,4.977986,7.419088,...,0.531348,0.536021,0.507245,0.510665,0.506654,0.492829,0.484946,0.462731,0.474542,0
3,10005,13.001012,6.376373,8.479070,10.935127,1.120146,10.528730,5.092280,7.257407,9.555641,...,0.484618,0.518575,0.480005,0.473664,0.502523,0.514538,0.537265,0.500549,0.485857,0
4,10006,2.751090,2.157247,4.778439,7.531138,2.043466,2.998565,2.410071,5.312022,11.679908,...,0.519394,0.478897,0.516000,0.495676,0.523968,0.515604,0.505154,0.516178,0.496556,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,10315,3.508636,2.245006,2.866946,3.807370,0.418680,4.266233,2.469076,3.211018,4.369106,...,0.493699,0.523106,0.478229,0.527235,0.465137,0.481502,0.487168,0.478506,0.496092,0
166,10316,5.668383,4.374410,2.166326,1.630367,0.352707,6.157412,4.673679,2.312181,1.822349,...,0.504561,0.494965,0.491063,0.470268,0.501390,0.504902,0.496936,0.495394,0.482235,0
167,10318,4.370899,2.288354,1.699240,5.647237,0.892734,6.248704,2.952553,2.143792,7.067242,...,0.459103,0.472405,0.458490,0.470130,0.497453,0.485832,0.499386,0.501177,0.508355,0
168,10319,12.263410,4.861399,13.168760,14.862353,1.869617,11.409593,4.538210,13.734878,10.770214,...,0.483272,0.509207,0.443675,0.472377,0.494690,0.491955,0.558415,0.510408,0.507811,0


In [8]:
dfec

,Unnamed: 0,Fp1a delta,Fp1a theta,Fp1a alpha,Fp1a beta,Fp1a gamma,Fp2a delta,Fp2a theta,Fp2a alpha,Fp2a beta,...,P3/P4 theta,P3/P4 alpha,P3/P4 beta,P3/P4 gamma,O1/O2 delta,O1/O2 theta,O1/O2 alpha,O1/O2 beta,O1/O2 gamma,label
0,10002,2.475371,1.638637,3.015637,11.598780,3.483368,2.282586,1.552220,2.664108,7.493583,...,0.500657,0.550677,0.566536,0.580211,0.495397,0.495227,0.520955,0.557818,0.472947,0
1,10003,19.922156,21.127609,39.019259,6.427448,0.734595,21.073421,22.050736,37.863523,6.997013,...,0.462132,0.490572,0.481299,0.490050,0.480509,0.494900,0.492222,0.510392,0.615860,0
2,10004,7.441585,7.143337,11.554174,6.022341,0.700233,7.537999,6.685701,9.016543,6.249960,...,0.535934,0.554881,0.513354,0.509120,0.502332,0.495862,0.464250,0.479605,0.473999,0
3,10005,9.808763,6.968096,11.118838,10.138833,0.358580,9.058049,6.610697,10.778979,9.745145,...,0.500293,0.470638,0.482173,0.483798,0.516039,0.523242,0.544121,0.498107,0.504212,0
4,10006,3.142933,4.254568,8.852245,7.044991,0.569409,3.266391,4.244158,8.766980,7.114891,...,0.507248,0.411856,0.471365,0.498517,0.514724,0.508148,0.471631,0.503539,0.503567,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,10315,4.020837,2.481817,4.422772,5.099212,0.509357,3.022170,1.806559,3.431501,3.663232,...,0.488744,0.356047,0.460454,0.484970,0.479487,0.487951,0.429576,0.509132,0.485781,0
171,10316,4.775194,3.587250,3.626048,1.975405,0.432544,5.345416,3.560776,4.097011,4.051232,...,0.479859,0.370586,0.459771,0.481945,0.513476,0.508598,0.381715,0.447990,0.487156,0
173,10318,7.095514,3.906354,4.303735,6.297427,0.758554,8.317032,4.403639,4.701117,7.391188,...,0.409012,0.343040,0.410707,0.478306,0.511121,0.488979,0.555404,0.515656,0.510222,0
174,10319,10.918286,5.050628,23.635058,10.075020,0.655482,11.342959,5.360507,27.146225,11.444452,...,0.488522,0.421483,0.440575,0.439214,0.517001,0.515320,0.514562,0.506926,0.504901,0


In [9]:
X_eo = dfeo.drop(['Unnamed: 0', 'label'], axis=1)
X_ec = dfec.drop(['Unnamed: 0', 'label'], axis=1)
y_eo = dfeo['label']
y_ec = dfeo['label']

> Saved models are retrieved below. The `create_model` function needs to be in place for the deep learning models to load correctly.

In [10]:
fitted_model = pickle.load(open('eeg_model.pkl', 'rb'))
fitted_model_ncv = pickle.load(open('eeg_model_ncv.pkl', 'rb'))
best_thr_ncv = pickle.load(open('eeg_best_thr_ncv.pkl', 'rb'))

In [11]:
def create_model(neurons1=100, neurons2=30, dropout1=0, dropout2=0, constraint=0):
    model = Sequential()
    model.add(Dense(neurons1, input_dim=150, activation='relu', kernel_constraint=max_norm(constraint)))
    model.add(Dropout(dropout1))
    model.add(Dense(neurons2, activation='relu', kernel_constraint=max_norm(constraint)))
    model.add(Dropout(dropout2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [12]:
fitted_dl_model = joblib.load('eeg_dl_model.pkl')
fitted_dl_model.named_steps['nn'].model = load_model('eeg_dl_model.h5')

In [13]:
fitted_dl_model_ncv = joblib.load('eeg_dl_model_ncv.pkl')
fitted_dl_model_ncv.named_steps['nn'].model = load_model('eeg_dl_model_ncv.h5')

> The various models are then tested on the two subsets of MPIL data and metrics are displayed. 
>
> The confusion matrix in each case is formatted as follows:  
> 
> &nbsp; &nbsp; \[[ true negatives | false positives ]  
> &nbsp; &nbsp; &nbsp;[ false negatives | true positives ]]  
> 
> The key variables of interest for clinical purposes are:
> - sensitivity (= class 1 recall) &ndash; the probablility of correctly identifying a positive (alcoholic) case
> - specificity (= class 0 recall) &ndash; the probablility of correctly identifying a negative (non-alcoholic) case
>
> Note that if sensitivity and specificity add up to 1, the predictor is making absolutely no distinction between positive and negative cases.

In [14]:
# MODEL 1 - L2 model, eyes open
pred = fitted_model.predict(X_eo)
print(confusion_matrix(y_eo, pred))
print(classification_report(y_eo, pred))

[[121  19]
 [ 26   2]]
              precision    recall  f1-score   support

           0       0.82      0.86      0.84       140
           1       0.10      0.07      0.08        28

    accuracy                           0.73       168
   macro avg       0.46      0.47      0.46       168
weighted avg       0.70      0.73      0.72       168



In [15]:
# MODEL 2 - Nested cross validation (NCV) derived L2 model, eyes open
pred = fitted_model_ncv.predict(X_eo)
print(confusion_matrix(y_eo, pred))
print(classification_report(y_eo, pred))

[[103  37]
 [ 22   6]]
              precision    recall  f1-score   support

           0       0.82      0.74      0.78       140
           1       0.14      0.21      0.17        28

    accuracy                           0.65       168
   macro avg       0.48      0.48      0.47       168
weighted avg       0.71      0.65      0.68       168



In [16]:
# MODEL 3 - NCV-derived L2 model with optimised threshold, eyes open
try:
    pred = fitted_model_ncv.predict_proba(X_eo)
    pred = pred[:, 1]
except:
    pred = fitted_model_ncv.decision_function(X_eo)
y_pred = [int(p > best_thr_ncv) for p in pred]
print(confusion_matrix(y_eo, y_pred))
print(classification_report(y_eo, y_pred))

[[100  40]
 [ 22   6]]
              precision    recall  f1-score   support

           0       0.82      0.71      0.76       140
           1       0.13      0.21      0.16        28

    accuracy                           0.63       168
   macro avg       0.48      0.46      0.46       168
weighted avg       0.70      0.63      0.66       168



In [17]:
# MODEL 4 - deep learning model, eyes open
pred = (fitted_dl_model.predict(X_eo) > 0.5).astype(int)
print(confusion_matrix(y_eo, pred))
print(classification_report(y_eo, pred))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[[80 60]
 [19  9]]
              precision    recall  f1-score   support

           0       0.81      0.57      0.67       140
           1       0.13      0.32      0.19        28

    accuracy                           0.53       168
   macro avg       0.47      0.45      0.43       168
weighted avg       0.70      0.53      0.59       168



In [18]:
# MODEL 5 - NCV-derived deep learning model, eyes open
pred = (fitted_dl_model_ncv.predict(X_eo) > 0.5).astype(int)
print(confusion_matrix(y_eo, pred))
print(classification_report(y_eo, pred))

[[138   2]
 [ 28   0]]
              precision    recall  f1-score   support

           0       0.83      0.99      0.90       140
           1       0.00      0.00      0.00        28

    accuracy                           0.82       168
   macro avg       0.42      0.49      0.45       168
weighted avg       0.69      0.82      0.75       168



In [19]:
# MODEL 6 - L2 model, eyes closed
pred = fitted_model.predict(X_ec)
print(confusion_matrix(y_ec, pred))
print(classification_report(y_ec, pred))

[[131   9]
 [ 28   0]]
              precision    recall  f1-score   support

           0       0.82      0.94      0.88       140
           1       0.00      0.00      0.00        28

    accuracy                           0.78       168
   macro avg       0.41      0.47      0.44       168
weighted avg       0.69      0.78      0.73       168



In [20]:
# MODEL 7 - NCV-derived L2 model, eyes closed
pred = fitted_model_ncv.predict(X_ec)
print(confusion_matrix(y_ec, pred))
print(classification_report(y_ec, pred))

[[111  29]
 [ 23   5]]
              precision    recall  f1-score   support

           0       0.83      0.79      0.81       140
           1       0.15      0.18      0.16        28

    accuracy                           0.69       168
   macro avg       0.49      0.49      0.49       168
weighted avg       0.71      0.69      0.70       168



In [21]:
# MODEL 8 - NCV-derived L2 model with optimised threshold, eyes closed
try:
    pred = fitted_model_ncv.predict_proba(X_ec)
    pred = pred[:, 1]
except:
    pred = fitted_model_ncv.decision_function(X_ec)
y_pred = [int(p > best_thr_ncv) for p in pred]
print(confusion_matrix(y_ec, y_pred))
print(classification_report(y_ec, y_pred))

[[111  29]
 [ 23   5]]
              precision    recall  f1-score   support

           0       0.83      0.79      0.81       140
           1       0.15      0.18      0.16        28

    accuracy                           0.69       168
   macro avg       0.49      0.49      0.49       168
weighted avg       0.71      0.69      0.70       168



In [22]:
# MODEL 9 - deep learning model, eyes closed
pred = (fitted_dl_model.predict(X_ec) > 0.5).astype(int)
print(confusion_matrix(y_ec, pred))
print(classification_report(y_ec, pred))

[[120  20]
 [ 23   5]]
              precision    recall  f1-score   support

           0       0.84      0.86      0.85       140
           1       0.20      0.18      0.19        28

    accuracy                           0.74       168
   macro avg       0.52      0.52      0.52       168
weighted avg       0.73      0.74      0.74       168



In [23]:
# MODEL 10 - NCV-derived deep learning model, eyes closed
pred = (fitted_dl_model_ncv.predict(X_ec) > 0.5).astype(int)
print(confusion_matrix(y_ec, pred))
print(classification_report(y_ec, pred))

[[122  18]
 [ 28   0]]
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       140
           1       0.00      0.00      0.00        28

    accuracy                           0.73       168
   macro avg       0.41      0.44      0.42       168
weighted avg       0.68      0.73      0.70       168



> **WITHOUT ABSOLUTE POWERS**
> 
> The MPIL datasets are modified by removing absolute powers, and these are then used for testing models trained on UCI data without absolute powers.

In [24]:
X_eo_no_ap = dfeo.drop(['Unnamed: 0', 'Fp1a delta', 'Fp1a theta', 'Fp1a alpha', 'Fp1a beta',
       'Fp1a gamma', 'Fp2a delta', 'Fp2a theta', 'Fp2a alpha',
       'Fp2a beta', 'Fp2a gamma', 'F3a delta', 'F3a theta', 'F3a alpha',
       'F3a beta', 'F3a gamma', 'F4a delta', 'F4a theta', 'F4a alpha',
       'F4a beta', 'F4a gamma', 'F7a delta', 'F7a theta', 'F7a alpha',
       'F7a beta', 'F7a gamma', 'F8a delta', 'F8a theta', 'F8a alpha',
       'F8a beta', 'F8a gamma', 'C3a delta', 'C3a theta', 'C3a alpha',
       'C3a beta', 'C3a gamma', 'C4a delta', 'C4a theta', 'C4a alpha',
       'C4a beta', 'C4a gamma', 'P3a delta', 'P3a theta', 'P3a alpha',
       'P3a beta', 'P3a gamma', 'P4a delta', 'P4a theta', 'P4a alpha',
       'P4a beta', 'P4a gamma', 'O1a delta', 'O1a theta', 'O1a alpha',
       'O1a beta', 'O1a gamma', 'O2a delta', 'O2a theta', 'O2a alpha',
       'O2a beta', 'O2a gamma', 'label'], axis=1)

In [25]:
X_ec_no_ap = dfec.drop(['Unnamed: 0', 'Fp1a delta', 'Fp1a theta', 'Fp1a alpha', 'Fp1a beta',
       'Fp1a gamma', 'Fp2a delta', 'Fp2a theta', 'Fp2a alpha',
       'Fp2a beta', 'Fp2a gamma', 'F3a delta', 'F3a theta', 'F3a alpha',
       'F3a beta', 'F3a gamma', 'F4a delta', 'F4a theta', 'F4a alpha',
       'F4a beta', 'F4a gamma', 'F7a delta', 'F7a theta', 'F7a alpha',
       'F7a beta', 'F7a gamma', 'F8a delta', 'F8a theta', 'F8a alpha',
       'F8a beta', 'F8a gamma', 'C3a delta', 'C3a theta', 'C3a alpha',
       'C3a beta', 'C3a gamma', 'C4a delta', 'C4a theta', 'C4a alpha',
       'C4a beta', 'C4a gamma', 'P3a delta', 'P3a theta', 'P3a alpha',
       'P3a beta', 'P3a gamma', 'P4a delta', 'P4a theta', 'P4a alpha',
       'P4a beta', 'P4a gamma', 'O1a delta', 'O1a theta', 'O1a alpha',
       'O1a beta', 'O1a gamma', 'O2a delta', 'O2a theta', 'O2a alpha',
       'O2a beta', 'O2a gamma', 'label'], axis=1)

In [26]:
fitted_model_no_ap = pickle.load(open('eeg_model_no_ap.pkl', 'rb'))
fitted_model_no_ap_ncv = pickle.load(open('eeg_model_no_ap_ncv.pkl', 'rb'))
best_thr_no_ap_ncv = pickle.load(open('eeg_best_thr_no_ap_ncv.pkl', 'rb'))

In [27]:
def create_model(neurons1=100, neurons2=30, dropout1=0, dropout2=0, constraint=0):
    model = Sequential()
    model.add(Dense(neurons1, input_dim=90, activation='relu', kernel_constraint=max_norm(constraint)))
    model.add(Dropout(dropout1))
    model.add(Dense(neurons2, activation='relu', kernel_constraint=max_norm(constraint)))
    model.add(Dropout(dropout2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [28]:
fitted_dl_model_no_ap = joblib.load('eeg_dl_model_no_ap.pkl')
fitted_dl_model_no_ap.named_steps['nn'].model = load_model('eeg_dl_model_no_ap.h5')

In [29]:
fitted_dl_model_no_ap_ncv = joblib.load('eeg_dl_model_no_ap_ncv.pkl')
fitted_dl_model_no_ap_ncv.named_steps['nn'].model = load_model('eeg_dl_model_no_ap_ncv.h5')

In [30]:
# MODEL 11 - linear SVM model, eyes open
pred = fitted_model_no_ap.predict(X_eo_no_ap)
print(confusion_matrix(y_eo, pred))
print(classification_report(y_eo, pred))

[[116  24]
 [ 23   5]]
              precision    recall  f1-score   support

           0       0.83      0.83      0.83       140
           1       0.17      0.18      0.18        28

    accuracy                           0.72       168
   macro avg       0.50      0.50      0.50       168
weighted avg       0.72      0.72      0.72       168



In [31]:
# MODEL 12 - NCV-derived L1 model, eyes open
pred = fitted_model_no_ap_ncv.predict(X_eo_no_ap)
print(confusion_matrix(y_eo, pred))
print(classification_report(y_eo, pred))

[[108  32]
 [ 24   4]]
              precision    recall  f1-score   support

           0       0.82      0.77      0.79       140
           1       0.11      0.14      0.12        28

    accuracy                           0.67       168
   macro avg       0.46      0.46      0.46       168
weighted avg       0.70      0.67      0.68       168



In [32]:
# MODEL 13 - NCV-derived L1 model with optimised threshold, eyes open
try:
    pred = fitted_model_no_ap_ncv.predict_proba(X_eo_no_ap)
    pred = pred[:, 1]
except:
    pred = fitted_model_no_ap_ncv.decision_function(X_eo_no_ap)
y_pred = [int(p > best_thr_ncv) for p in pred]
print(confusion_matrix(y_eo, y_pred))
print(classification_report(y_eo, y_pred))

[[107  33]
 [ 24   4]]
              precision    recall  f1-score   support

           0       0.82      0.76      0.79       140
           1       0.11      0.14      0.12        28

    accuracy                           0.66       168
   macro avg       0.46      0.45      0.46       168
weighted avg       0.70      0.66      0.68       168



In [33]:
# MODEL 14 - deep learning model, eyes open
pred = (fitted_dl_model_no_ap.predict(X_eo_no_ap) > 0.5).astype(int)
print(confusion_matrix(y_eo, pred))
print(classification_report(y_eo, pred))

[[65 75]
 [11 17]]
              precision    recall  f1-score   support

           0       0.86      0.46      0.60       140
           1       0.18      0.61      0.28        28

    accuracy                           0.49       168
   macro avg       0.52      0.54      0.44       168
weighted avg       0.74      0.49      0.55       168



In [34]:
# MODEL 15 - NCV-derived deep learning model, eyes open
# This has slight utility as a rule-out test
pred = (fitted_dl_model_no_ap_ncv.predict(X_eo_no_ap) > 0.5).astype(int)
print(confusion_matrix(y_eo, pred))
print(classification_report(y_eo, pred))

[[ 30 110]
 [  3  25]]
              precision    recall  f1-score   support

           0       0.91      0.21      0.35       140
           1       0.19      0.89      0.31        28

    accuracy                           0.33       168
   macro avg       0.55      0.55      0.33       168
weighted avg       0.79      0.33      0.34       168



In [35]:
# MODEL 16 - linear SVM model, eyes closed
pred = fitted_model_no_ap.predict(X_ec_no_ap)
print(confusion_matrix(y_ec, pred))
print(classification_report(y_ec, pred))

[[128  12]
 [ 28   0]]
              precision    recall  f1-score   support

           0       0.82      0.91      0.86       140
           1       0.00      0.00      0.00        28

    accuracy                           0.76       168
   macro avg       0.41      0.46      0.43       168
weighted avg       0.68      0.76      0.72       168



In [36]:
# MODEL 17 - NCV-derived L1 model, eyes closed
pred = fitted_model_no_ap_ncv.predict(X_ec_no_ap)
print(confusion_matrix(y_ec, pred))
print(classification_report(y_ec, pred))

[[113  27]
 [ 25   3]]
              precision    recall  f1-score   support

           0       0.82      0.81      0.81       140
           1       0.10      0.11      0.10        28

    accuracy                           0.69       168
   macro avg       0.46      0.46      0.46       168
weighted avg       0.70      0.69      0.69       168



In [37]:
# MODEL 18 - NCV-derived L1 model with optimised threshold, eyes closed
try:
    pred = fitted_model_no_ap_ncv.predict_proba(X_ec_no_ap)
    pred = pred[:, 1]
except:
    pred = fitted_model_no_ap_ncv.decision_function(X_ec_no_ap)
y_pred = [int(p > best_thr_ncv) for p in pred]
print(confusion_matrix(y_ec, y_pred))
print(classification_report(y_ec, y_pred))

[[112  28]
 [ 24   4]]
              precision    recall  f1-score   support

           0       0.82      0.80      0.81       140
           1       0.12      0.14      0.13        28

    accuracy                           0.69       168
   macro avg       0.47      0.47      0.47       168
weighted avg       0.71      0.69      0.70       168



In [38]:
# MODEL 19 - deep learning model, eyes closed
pred = (fitted_dl_model_no_ap.predict(X_ec_no_ap) > 0.5).astype(int)
print(confusion_matrix(y_ec, pred))
print(classification_report(y_ec, pred))

[[86 54]
 [14 14]]
              precision    recall  f1-score   support

           0       0.86      0.61      0.72       140
           1       0.21      0.50      0.29        28

    accuracy                           0.60       168
   macro avg       0.53      0.56      0.50       168
weighted avg       0.75      0.60      0.65       168



In [39]:
# MODEL 20 - NCV-derived deep learning model, eyes closed
pred = (fitted_dl_model_no_ap_ncv.predict(X_ec_no_ap) > 0.5).astype(int)
print(confusion_matrix(y_ec, pred))
print(classification_report(y_ec, pred))

[[56 84]
 [ 8 20]]
              precision    recall  f1-score   support

           0       0.88      0.40      0.55       140
           1       0.19      0.71      0.30        28

    accuracy                           0.45       168
   macro avg       0.53      0.56      0.43       168
weighted avg       0.76      0.45      0.51       168



> Above we have 10 different models each tested on 2 datasets in order to give 20 tests in total. In most of the cases sensitivity and specificity add up to less than 1, meaning that the usefulness of the predictor is worse than a coin toss. A positive result for an individual subject from any of these predictors is of no real value, as the highest positive predictive value (class 1 precision, indicating the probability that a positive result is correct) is 0.21. Note that in the test sets we are using, the proportion of alcoholic subjects is 0.17, so the negative predictive value (class 0 precision) of a completely random coin toss as a test for alcoholism in this subject group (i.e. the chance of a *negative* result being correct) would be 0.83. The only models with a negative predictive higher than this are some of the deep learning models.
>
> Model 15 would have slight utility as a rule-out test. Class 0 precision is 0.91, meaning that 91% of negative test results would be correct, whereas with a class 1 precision of only 0.19, a positive test result would be of no real use. Recall values for the two classes show that 89% of hazardous drinkers are identified by the test as such, but then so are 79% of people who are not hazardous drinkers. The usefulness of model 15 as a predictor should be interpreted very cautiously, as one seemingly helpful result out of 20 tests could easily occur by chance.
>
> Overall, the models trained on the UCI dataset have not proven helpful for predicting hazardous drinking when tested on the MPIL dataset. It may still be possible to train some helpful models for this purpose &ndash; we don't know for sure either way &ndash; but this might require a different approach to calculating band powers, an approach other than band power, more rigorous artefact removal, or a test set with testing conditions and a case definition more closely matching those of the training set.